In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('Check.xlsx')

In [4]:
for i in df['Label From History'].unique():
    print(i)
    print(df[df['Label From History'] == i].label.value_counts())

nan
Series([], Name: label, dtype: int64)
Low
1    645
0    232
2     50
Name: label, dtype: int64
FreeRoll
1    562
0     43
2      7
Name: label, dtype: int64
High
0    1
Name: label, dtype: int64
Medium
2    3
0    1
Name: label, dtype: int64


In [21]:
import pyodbc
cnx: pyodbc.Connection = pyodbc.connect(
"Driver={ODBC Driver 17 for SQL Server};"
"Server=DWH;"
"Database=dwOper;"
"Trusted_Connection=yes;")
query = f"""
SELECT a.TOTOID,
    SUM(a.DepositCount) AS DepositCount,
    SUM(a.DepositAmount) AS DepositAmount,
    MAX(a.Age) AS Age,
    a.Timeline
FROM
(SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    MAX(DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))) AS Age,
    CASE
    WHEN cu.RegistrationDate <= '2020-01-01' THEN 'old'
    WHEN cu.RegistrationDate <= '2021-01-01' THEN '3 year new'
    ELSE '2 year new'
END AS Timeline
FROM Payment p
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
    AND p.PaymentStatusID = 8
    AND p.PaymentTypeID = 2
    AND sp.PaymentSystemName NOT IN ('BONUS', 'CasinoTransfer', 'Casino Transfer', 'PokerTransfer', 'TRANSFER')
GROUP BY cu.PartnerUserID, cu.RegistrationDate
UNION
SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    MAX(DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))) AS Age,
    CASE
    WHEN cu.RegistrationDate <= '2020-01-01' THEN 'old'
    WHEN cu.RegistrationDate <= '2021-01-01' THEN '3 year new'
    ELSE '2 year new'
    END AS Timeline
    FROM Payment p
INNER JOIN VIEW_sport_PartnerUser_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
    AND p.PaymentStatusID = 8
    AND p.PaymentTypeID = 2
    AND sp.PaymentSystemName NOT IN ('BONUS', 'CasinoTransfer', 'Casino Transfer', 'PokerTransfer', 'TRANSFER')
GROUP BY cu.PartnerUserID, cu.RegistrationDate) a
GROUP BY a.TOTOID, a.Timeline
ORDER BY TOTOID
"""
df = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

C:\Users\narek.meloyan\AppData\Local\Temp\ipykernel_12332\2324966188.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx, index_col='TOTOID')


In [22]:
df_new_2 = df[df.Timeline == '2 year new']
df_new_3 = df[df.Timeline == '3 year new']
df_old = df[df.Timeline == 'old']
print(np.mean(df_old['DepositAmount']), np.mean(df_old['DepositCount']), np.std(df_old['DepositAmount']), np.std(df_old['DepositCount']))
print(np.mean(df_new_3['DepositAmount']), np.mean(df_new_3['DepositCount']), np.std(df_new_3['DepositAmount']), np.std(df_new_3['DepositCount']))
print(np.mean(df_new_2['DepositAmount']), np.mean(df_new_2['DepositCount']), np.std(df_new_2['DepositAmount']), np.std(df_new_2['DepositCount']))


133352.1556430061 30.10705869288949 1813387.7108707326 79.00654906021961
186186.34500962947 45.52260868133858 3913725.5033901553 110.09555787042555
210412.8746777991 56.72458492254494 2502040.570997709 122.48013724767408


#### From here we see that old and new customers' mean differs according to deposit count and amount
###### So for purposes of balancing and getting more precise result will be more efficient to pick users that registered relatively close(In this case it becomes over 3 years period which is normal measure at statistical sight)

In [14]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
SELECT cu.PartnerUserID                                      AS TOTOID,
       cu.UserID                                             AS UserID,
       DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE)) AS Age,
       IIF(CAST(cu.RegistrationDate AS DATE) > CAST(su.RegistrationDate AS DATE), CAST(su.RegistrationDate AS DATE),
           CAST(cu.RegistrationDate AS DATE))                AS RegDate
FROM VIEW_PlatformPartnerUsers_TotogamingAm cu
    LEFT JOIN VIEW_sport_PartnerUser_TotogamingAm su ON cu.PartnerUserId = su.PartnerUserId
WHERE cu.RegistrationDate >= '2020-01-01'
  AND DATEDIFF(DAY, CAST(su.RegistrationDate AS DATE), CAST(cu.RegistrationDate AS DATE)) <= 5
  AND DATEDIFF(DAY, CAST(su.RegistrationDate AS DATE), CAST(cu.RegistrationDate AS DATE)) >= -5
GROUP BY cu.PartnerUserID, cu.UserID, cu.RegistrationDate, su.RegistrationDate, DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))
"""
users_3_years = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

C:\Users\narek.meloyan\AppData\Local\Temp\ipykernel_12332\2348837927.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users_3_years = pd.read_sql(query, cnx, index_col='TOTOID')


In [15]:
users_3_years

,UserID,Age,RegDate
TOTOID,,,
100738319,10556123,43,2020-01-24
100756775,11301624,32,2020-03-17
100987885,12254546,43,2020-06-26
101020642,13913613,26,2020-10-28
101028645,14628656,25,2020-12-04
...,...,...,...
101176305,23518934,26,2022-01-23
101127295,19869165,26,2021-08-23
101043287,15297779,46,2021-01-11


In [16]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
SELECT cu.PartnerUserID                                      AS TOTOID,
       cu.UserID                                             AS UserID,
       DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE)) AS Age,
       IIF(CAST(cu.RegistrationDate AS DATE) > CAST(su.RegistrationDate AS DATE), CAST(su.RegistrationDate AS DATE),
           CAST(cu.RegistrationDate AS DATE))                AS RegDate
FROM VIEW_PlatformPartnerUsers_TotogamingAm cu
    LEFT JOIN VIEW_sport_PartnerUser_TotogamingAm su ON cu.PartnerUserId = su.PartnerUserId
WHERE cu.RegistrationDate >= '2021-01-01'
  AND DATEDIFF(DAY, CAST(su.RegistrationDate AS DATE), CAST(cu.RegistrationDate AS DATE)) <= 5
  AND DATEDIFF(DAY, CAST(su.RegistrationDate AS DATE), CAST(cu.RegistrationDate AS DATE)) >= -5
GROUP BY cu.PartnerUserID, cu.UserID, cu.RegistrationDate, su.RegistrationDate, DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))
"""
users_2_years = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

C:\Users\narek.meloyan\AppData\Local\Temp\ipykernel_12332\3938028672.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users_2_years = pd.read_sql(query, cnx, index_col='TOTOID')


In [17]:
users_2_years

,UserID,Age,RegDate
TOTOID,,,
101042653,15277121,61,2021-01-10
101071813,16402507,43,2021-03-08
101090251,17159464,38,2021-04-14
101118244,19126297,44,2021-07-19
101128442,19947246,30,2021-08-27
...,...,...,...
101130273,20113053,28,2021-09-04
101127295,19869165,26,2021-08-23
101095966,17450122,34,2021-04-29
